In [3]:
#Import libraries
import sys
import os
import pandas as pd
import mmh3
import numpy as np

We start by defining the necessary functions to get document similarity (week 5 of course exercises).

In [76]:
# create shingle function
def shingles(string:str,q:int):
    output = set()
    for i in range(len(string)+1):
        if i < q:
            pass
        else:
            output.add(' '.join(string[i-q:i]))
    return output

#create listhash function
def listhash(l,seeds):
    vals = set()

    for e in l:
        val = 0
        for seed in seeds:
            val = val ^ mmh3.hash(e, seed)
        vals.add(val)
    return vals

#create signatures function
def signatures(docs, q=9, k=20):
    sign = {}
    for key, value in docs.items():
        sign[key] = listhash(shingles(value,q=q),np.arange(k))
    return sign

#create jaccard sim function
def jaccard(doc1, doc2):
    doc1=set(doc1)
    doc2=set(doc2)
    intersect = doc1.intersection(doc2)
    union = doc1.union(doc2)
    if len(union) != 0:
        return len(intersect) / len(union)
    else:
        return 0
    
#create document sim function
def similarity(docs:dict):
    output = np.zeros((len(docs.keys()),len(docs.keys())))
    for key1, value1 in docs.items():
        for key2, value2 in docs.items():
            if key1 == key2:
                pass
            else:
                jac_value = jaccard(value1,value2)
                output[key1,key2] = jac_value
    return output
            

Load email data!

In [275]:
from ast import literal_eval
emails = pd.read_csv('data/clean_spam.csv', encoding='latin')
emails.tokens = emails.tokens.apply(literal_eval) #convert tokens to list of tokens

#determine q and k
q = 5 #number of characters in each shingle
k = 20 #number of hashes per shingle

In [276]:
#create signatures for emails (we keep count based on index in emails)
email_signatures = {emails.index[i]: listhash(shingles(emails['text'][i], q=q),np.arange(k)) for i in emails.index}

In [277]:
#create signatures for emails
email_similarity = similarity(email_signatures)

In [396]:
def weighted_knn(x, y_train,test_idx,low,high,k_neighbours=5):
    y_test = []
    mask = np.ones(len(x),bool)
    mask[y_test] = False
    for i in test_idx:
        ind = []
        temp = np.argpartition(x[i], -k_neighbours)[-k_neighbours:]
        temp = np.flip(temp)
        for idx in temp:
            if idx>=high or idx < low:
                ind.append(idx)
        topk = x[i][ind]
        labels = {j: y_train[j] for j in ind}
        ham = 0
        spam = 0
        for key, value in labels.items():
            if value == 'ham':
                ham += x[i][key]
            if value == 'spam':
                spam += x[i][key]
        if ham>spam:
            y_test.append('ham')
        if ham == spam:
            y_test.append('ham')
        if ham<spam:
            y_test.append('spam')
    return y_test


In [439]:
from sklearn.metrics import f1_score
kfold=5
f1_scores = []
test_set_percent = ((len(emails)/kfold)/len(emails))
test_size = round(test_set_percent*len(emails))
former_test_idx = 0
y_pred1 = []
for i in range(kfold):
    y_test = emails['label'][former_test_idx:(i+1)*test_size]
    y_idx = y_test.index
    mask = np.ones(len(emails), bool)
    mask[y_idx] = False
    y_train = emails['label'][mask]
    y_pred = weighted_knn(email_similarity,y_train,y_idx,former_test_idx,(i+1)*test_size,5)
    y_pred1 = y_pred1 + list(y_pred)
    f1_scores.append(f1_score(y_test,y_pred, pos_label='spam'))
    former_test_idx += test_size
print(f'Average f1-score: {round(np.mean(f1_scores),2)}')

Average f1-score: 0.94


Lets try the same with bag of words instead of signatures:

In [434]:
#get bag of words as sets for each email
email_bow = {emails.index[i]: set(emails['tokens'][i]) for i in range(len(emails))}

In [435]:
#get similarity between documents
email_bow_sim = similarity(email_bow)

In [438]:
from sklearn.metrics import f1_score
kfold=5
f1_scores = []
test_set_percent = ((len(emails)/kfold)/len(emails))
test_size = round(test_set_percent*len(emails))
former_test_idx = 0
y_tests = []
y_pred2 = []
for i in range(kfold):
    y_test = emails['label'][former_test_idx:(i+1)*test_size]
    y_tests = y_tests + list(y_test)
    y_idx = y_test.index
    mask = np.ones(len(emails), bool)
    mask[y_idx] = False
    y_train = emails['label'][mask]
    y_pred = weighted_knn(email_bow_sim,y_train,y_idx,former_test_idx,(i+1)*test_size,5)
    y_pred2 = y_pred2 + list(y_pred)
    f1_scores.append(f1_score(y_test,y_pred, pos_label='spam'))
    former_test_idx += test_size
print(f'Average f1-score: {round(np.mean(f1_scores),2)}')

Average f1-score: 0.91


So, the model with signatures performs better, but is this significant? We will perform a McNemar test to evaluate this.

In [457]:
import scipy
#define McNemar test (inspired by a function made in Introduction to Machine Learning)
def mcnemar(y_true, y_pred1, y_pred2, alpha=0.05):
    nn = np.zeros((2,2))
    c1 = y_pred1 - y_true == 0
    c2 = y_pred2 - y_true == 0

    nn[0,0] = sum(c1 & c2)
    nn[0,1] = sum(c1 & ~c2)
    nn[1,0] = sum(~c1 & c2)
    nn[1,1] = sum(~c1 & ~c2)

    n = sum(nn.flat);
    n12 = nn[0,1]
    n21 = nn[1,0]

    thetahat = (n12-n21)/n
    Etheta = thetahat

    Q = n**2 * (n+1) * (Etheta+1) * (1-Etheta) / ( (n*(n12+n21) - (n12-n21)**2) )

    p = (Etheta + 1)*0.5 * (Q-1)
    q = (1-Etheta)*0.5 * (Q-1)

    CI = tuple(lm * 2 - 1 for lm in scipy.stats.beta.interval(1-alpha, a=p, b=q) )

    p = 2*scipy.stats.binom.cdf(min([n12,n21]), n=n12+n21, p=0.5)
    print(r'Result of McNemars test using $\alpha$ = ', alpha)
    print('Comparison matrix n')
    print(nn)
    if n12+n21 <= 10:
        print('Warning, n12+n21 is low: n12+n21=',(n12+n21))
    print(r'$\theta_hat$: ',thetahat)
    print(r'Approximate 1-$\alpha$ confidence interval of $\theta$: [$\theta_L$,$\theta_U$] = ', CI)
    print(r'p-value for two-sided test model 1 and model 2 have same accuracy (exact binomial test): p = ', p)

    return thetahat, CI, p 

Because of the design of this function, we need to vstack all y_test and y_pred to accomodate for all folds in the cross validation:

In [459]:
np_y_tests = np.zeros(len(y_tests))
np_y_pred1 = np.zeros(len(y_tests))
np_y_pred2 = np.zeros(len(y_tests))

for i in range(len(y_tests)):
    if y_tests[i] == 'spam':
        np_y_tests[i] = 1
    if y_pred1[i] == 'spam':
        np_y_pred1[i] = 1
    if y_pred2[i] == 'spam':
        np_y_pred2[i] = 1


mcresults = mcnemar(np_y_tests, np_y_pred1, np_y_pred2, alpha=0.05)

Result of McNemars test using $\alpha$ =  0.05
Comparison matrix n
[[5408.   75.]
 [  37.   50.]]
$\theta_hat$:  0.006822262118491921
Approximate 1-$\alpha$ confidence interval of $\theta$: [$\theta_L$,$\theta_U$] =  (0.003102931217386473, 0.010541499940176058)
p-value for two-sided test model 1 and model 2 have same accuracy (exact binomial test): p =  0.0004209679285168773


You can see that the p-value is very much lower than our level of confidence (alpha) of 0.05. Therefore, we can reject the null-hypothesis that the accuracies of the two models are equal!

Translated to English, this means that the first model based on signatures is significantly better than the model based on tokens.